In [ ]:
import warnings
warnings.filterwarnings('ignore')

##popularity

###data Process

In [ ]:
import pandas as pd

In [ ]:
booksData = pd.read_csv("/content/drive/MyDrive/Kuggle/Books.csv")
usersData = pd.read_csv("/content/drive/MyDrive/Kuggle/Users.csv")
ratingsData = pd.read_csv("/content/drive/MyDrive/Kuggle/Ratings.csv")

In [ ]:
booksData=booksData.drop(['Publisher', 'Image-URL-S','Book-Author','Image-URL-M','Image-URL-L'], axis=1)
booksData.head()

,ISBN,Book-Title,Year-Of-Publication
0,0195153448,Classical Mythology,2002
1,0002005018,Clara Callan,2001
2,0060973129,Decision in Normandy,1991
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,1999
4,0393045218,The Mummies of Urumchi,1999


In [ ]:
ratings_books = ratingsData.merge(booksData, on ='ISBN', how='left')
ratings_books.rename(columns = {'User-ID':'userId', 'ISBN':'bookID','Book-Rating':'rating'}, inplace = True)
ratings_books.head(2)

,userId,bookID,rating,Book-Title,Year-Of-Publication
0,276725,034545104X,0,Flesh Tones: A Novel,2002
1,276726,0155061224,5,Rites of Passage,2001


In [ ]:
ratings_books=ratings_books[ratings_books['Year-Of-Publication'].notna()]
ratings_books['Year-Of-Publication'].isna().sum()
ratings_books['Year-Of-Publication']=pd.to_numeric(ratings_books['Year-Of-Publication'], 'coerce').fillna(-1, downcast = 'infer')
ratings_books.isna().sum()


userId                 0
bookID                 0
rating                 0
Book-Title             0
Year-Of-Publication    0
dtype: int64

In [ ]:
ratings_books['rating']=ratings_books['rating']/2
ratings_books=ratings_books.iloc[:100000,0:]
ratings_books

,userId,bookID,rating,Book-Title,Year-Of-Publication
0,276725,034545104X,0.0,Flesh Tones: A Novel,2002
1,276726,0155061224,2.5,Rites of Passage,2001
2,276727,0446520802,0.0,The Notebook,1996
3,276729,052165615X,1.5,Help!: Level 1,1999
4,276729,0521795028,3.0,The Amsterdam Connection : Level 4 (Cambridge ...,2001
...,...,...,...,...,...
113830,26516,0373244479,4.5,Marked For Marriage (Montana Mavericks) (Silho...,2002
113831,26516,0373244487,0.0,Cordina's Crown Jewel,2002
113832,26516,0373244835,5.0,Her Montana Man (Montana Mavericks) (Silhouet...,2002
113833,26516,0373244916,0.0,Big Sky Cowboy (Montana Mavericks) (Silhouett...,2002


In [ ]:
threshold_date = '2003' 
# ratings_books['Year-Of-Publication']=pd.to_numeric(ratings_books['Year-Of-Publication'], errors='coerce')
ids = ratings_books[ratings_books['Year-Of-Publication'] < int(threshold_date)]['bookID'].values

training_data = ratings_books[ratings_books['bookID'].isin(ids)]
print(f'Training data size: {training_data.shape}')
testing_data = ratings_books[~ratings_books['bookID'].isin(ids)]
print(f'Testing data size: {testing_data.shape}')
testing_data

Training data size: (91274, 5)
Testing data size: (8726, 5)


,userId,bookID,rating,Book-Title,Year-Of-Publication
16,276747,0060517794,4.5,Little Altars Everywhere,2003
32,276762,0380000059,0.0,CLOUT,2024
35,276762,0451454952,0.0,House of the Sun (Shadowrun),2003
50,276762,3453877241,0.0,Live aus Bagdad. Das Tagebuch einer Kriegs-Rep...,2003
74,276786,842333533X,0.0,Vieja Nueva York,2003
...,...,...,...,...,...
113775,26516,0312289723,0.0,Ten Big Ones: A Stephanie Plum Novel,2004
113788,26516,0333654331,0.0,A History of Ireland,2003
113801,26516,0345443721,0.0,Starfist Kingdom's Fury (Starfist (Paperback)),2003
113802,26516,0345447069,0.0,Walking to Canterbury: A Modern Journey Throug...,2003


In [ ]:
testing_data.rename({'User-ID':'userId', 'ISBN':'bookID','Book-Rating':'rating'}, axis=1, inplace = True)

In [ ]:
def get_favorite_books(user_id, ratings_df):
    favorites = ratings_df[(ratings_df['userId'] == user_id) & (ratings_df['rating'] >= 3.5)].sort_values(by='rating', ascending=False)['bookID']
    return set(favorites if type(favorites) == pd.Series else [favorites])

In [ ]:
class ModelEvaluator:
  def __init__(self, training_data, testing_data, threshold=3.5):
    self.training_data = training_data
    self.testing_data = testing_data
    print(testing_data.columns)
    self.threshold = threshold

  def evaluate_model_for_user(self, model, user_id):
      #Getting the items in test set
      favorites_in_test = get_favorite_books(user_id, self.testing_data)
      
      #Getting a ranked recommendation list from a model for a given user (bookID, predicted_rating)
      person_recs_df = model.recommend_items(user_id, 
                                              items_to_ignore=get_favorite_books(user_id, self.training_data))
      
      # get only book with predicated rating >= 3.5
      person_recs_df = person_recs_df[person_recs_df['predicted_rating'] >= self.threshold].sort_values(by='predicted_rating', ascending=False)
      true_relevent = person_recs_df[person_recs_df['bookID'].isin(favorites_in_test)].shape[0]

      top_5_recommended = person_recs_df.head(5)
      top_10_recommended = person_recs_df.head(10)

      # Number of relevant and recommended items in top k
      hits_at_5_count = top_5_recommended[top_5_recommended['bookID'].isin(favorites_in_test)].shape[0]
      hits_at_10_count = top_10_recommended[top_10_recommended['bookID'].isin(favorites_in_test)].shape[0]

      precision_at_5 = hits_at_5_count / top_5_recommended.shape[0] if top_5_recommended.shape[0] != 0 else 1
      recall_at_5 = hits_at_5_count / true_relevent if true_relevent != 0 else 1

      precision_at_10 = hits_at_10_count / top_10_recommended.shape[0] if top_10_recommended.shape[0] != 0 else 1
      recall_at_10 = hits_at_10_count / true_relevent if true_relevent != 0 else 1

      person_metrics = {'hits@5_count':hits_at_5_count, 
                        'hits@10_count':hits_at_10_count,
                        'recommended@5_count':top_5_recommended.shape[0], 
                        'recommended@10_count':top_10_recommended.shape[0],  
                        'relevents': true_relevent,
                        'recall@5': recall_at_5,
                        'recall@10': recall_at_10,
                        'precision@5': precision_at_5,
                        'precision@10': precision_at_10}
      return person_metrics


  def evaluate_model(self, model):
      #print('Running evaluation for users')
      users_metrics = []
      print(self.testing_data.columns)
      users_ids = list(set(self.testing_data['userId'].values))
      for idx, user_id in enumerate(users_ids):
          metrics = self.evaluate_model_for_user(model, user_id)  
          metrics['userId'] = user_id
          users_metrics.append(metrics)
      print('%d users processed' % idx)

      detailed_results_df = pd.DataFrame(users_metrics) \
                          .sort_values('hits@5_count', ascending=False)
      
      global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['relevents'].sum())
      global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['relevents'].sum())
      
      global_precision_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['recommended@5_count'].sum())
      global_precision_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['recommended@10_count'].sum())

      global_metrics = {'modelName': model.get_model_name(),
                        'recall@5': global_recall_at_5,
                        'recall@10': global_recall_at_10,
                        'precision@5': global_precision_at_5,
                        'precision@10': global_precision_at_10}    
      return global_metrics, detailed_results_df
    
model_evaluator = ModelEvaluator(training_data, testing_data)              

Index(['userId', 'bookID', 'rating', 'Book-Title', 'books_tags', 'Book-Author',
       'Year-Of-Publication'],
      dtype='object')


In [ ]:
populartiy = testing_data.groupby('bookID').agg({'rating': ['mean', 'count']}).reset_index()
populartiy.columns = ['bookID', 'ratings_mean', 'ratings_count']
populartiy.sort_values(by='ratings_mean', ascending=False)

,bookID,ratings_mean,ratings_count
2928,0767909208,5.0,1
3878,1591450365,5.0,1
248,006057898X,5.0,1
563,0312319517,5.0,1
565,0312320248,5.0,1
...,...,...,...
3005,0786016418,0.0,1
3003,0786016051,0.0,1
3002,0786015993,0.0,1
3001,0786015780,0.0,1


###Training

In [ ]:
class PopularityRecommender:
  
  MODEL_NAME = 'Popularity'
  
  def __init__(self, popularities_df):
      self.popularities_df = popularities_df
      
  def get_model_name(self):
      return self.MODEL_NAME

  def weighted_rating(self, x, m, C):
    v = x['ratings_count']
    R = x['ratings_mean']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

  def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
      C = self.popularities_df['ratings_mean'].mean()

      self.popularities_df['predicted_rating'] = self.popularities_df.apply(lambda x: self.weighted_rating(x, 3.5, C), axis=1)

      recommendations_df = self.popularities_df[~self.popularities_df['bookID'].isin(items_to_ignore)] \
                              .sort_values('predicted_rating', ascending = False) \
                              .head(topn)

      return recommendations_df
    
popularity_model = PopularityRecommender(populartiy)

In [ ]:
popularity_model.popularities_df

,bookID,ratings_mean,ratings_count
0,0002005395,2.0,2
1,0006392466,0.0,1
2,0006498000,0.0,1
3,0006552390,2.5,1
4,0007104758,0.0,1
...,...,...,...
4563,9509009059,1.5,3
4564,9810455674,3.5,1
4565,9871106181,0.0,1
4566,9875021245,0.0,1


In [ ]:
print('Evaluating Popularity recommendation model...')
pop_global_metrics, pop_detailed_results_df = model_evaluator.evaluate_model(popularity_model)
print('\nGlobal metrics:\n%s' % pop_global_metrics)
pop_detailed_results_df.head(10)

Evaluating Popularity recommendation model...
Index(['userId', 'bookID', 'rating', 'Book-Title', 'Year-Of-Publication'], dtype='object')
1943 users processed

Global metrics:
{'modelName': 'Popularity', 'recall@5': nan, 'recall@10': nan, 'precision@5': nan, 'precision@10': nan}


,hits@5_count,hits@10_count,recommended@5_count,recommended@10_count,relevents,recall@5,recall@10,precision@5,precision@10,userId
0,0,0,0,0,0,1,1,1,1,8192
1276,0,0,0,0,0,1,1,1,1,5526
1304,0,0,0,0,0,1,1,1,1,22062
1303,0,0,0,0,0,1,1,1,1,5674
1302,0,0,0,0,0,1,1,1,1,5665
1301,0,0,0,0,0,1,1,1,1,13850
1300,0,0,0,0,0,1,1,1,1,13835
1299,0,0,0,0,0,1,1,1,1,22023
1298,0,0,0,0,0,1,1,1,1,22019
1297,0,0,0,0,0,1,1,1,1,22012


In [ ]:
popularity_model.recommend_items(5)



,bookID,ratings_mean,ratings_count,predicted_rating
3398,1400032717,4.111111,9,3.446656
4028,1844262553,4.285714,7,3.436496
1505,0399149821,4.333333,6,3.377180
2054,0451526341,3.714286,14,3.319040
2640,0743454529,5.000000,3,3.243570
706,0316710571,4.500000,4,3.211094
559,0312316690,4.666667,3,3.089724
1819,0446531421,4.666667,3,3.089724
746,0345410998,4.250000,4,3.077761
297,0061015725,3.366667,15,3.058552


##Content-based book-auth

###pre processing data


In [ ]:
booksData = pd.read_csv("/content/drive/MyDrive/Kuggle/Books.csv")
usersData = pd.read_csv("/content/drive/MyDrive/Kuggle/Users.csv")
ratingsData = pd.read_csv("/content/drive/MyDrive/Kuggle/Ratings.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
booksData=booksData.drop(['Publisher', 'Image-URL-S','Image-URL-M','Image-URL-L'], axis=1)

booksData

,ISBN,Book-Title,Book-Author,Year-Of-Publication
0,0195153448,Classical Mythology,Mark P. O. Morford,2002
1,0002005018,Clara Callan,Richard Bruce Wright,2001
2,0060973129,Decision in Normandy,Carlo D'Este,1991
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999
...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988
271356,0525447644,From One to One Hundred,Teri Sloat,1991
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004
271358,0192126040,Republic (World's Classics),Plato,1996


In [ ]:
def clean(x):
    return x.replace(" ","").lower()

In [ ]:
books_tags=[]
for i in range(len(booksData['Book-Title'])):
     
    title=clean(str(booksData['Book-Title'][i]))
    bookauth=clean(str(booksData['Book-Author'][i]))
    books_tags.append(title+" "+bookauth ) 

In [ ]:
booksData.insert(2, "books_tags", books_tags, True)
booksData

,ISBN,Book-Title,books_tags,Book-Author,Year-Of-Publication
0,0195153448,Classical Mythology,classicalmythology markp.o.morford,Mark P. O. Morford,2002
1,0002005018,Clara Callan,claracallan richardbrucewright,Richard Bruce Wright,2001
2,0060973129,Decision in Normandy,decisioninnormandy carlod'este,Carlo D'Este,1991
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,flu:thestoryofthegreatinfluenzapandemicof1918a...,Gina Bari Kolata,1999
4,0393045218,The Mummies of Urumchi,themummiesofurumchi e.j.w.barber,E. J. W. Barber,1999
...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,there'sabatinbunkfive pauladanziger,Paula Danziger,1988
271356,0525447644,From One to One Hundred,fromonetoonehundred terisloat,Teri Sloat,1991
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,lilydale:thetruestoryofthetownthattalkstothede...,Christine Wicker,2004
271358,0192126040,Republic (World's Classics),republic(world'sclassics) plato,Plato,1996


In [ ]:
booksData['books_tags']

0                        classicalmythology markp.o.morford
1                            claracallan richardbrucewright
2                            decisioninnormandy carlod'este
3         flu:thestoryofthegreatinfluenzapandemicof1918a...
4                          themummiesofurumchi e.j.w.barber
                                ...                        
271355                  there'sabatinbunkfive pauladanziger
271356                        fromonetoonehundred terisloat
271357    lilydale:thetruestoryofthetownthattalkstothede...
271358                      republic(world'sclassics) plato
271359    aguidedtourofrenedescartes'meditationsonfirstp...
Name: books_tags, Length: 271360, dtype: object

In [ ]:
ratings_books = ratingsData.merge(booksData, on ='ISBN', how='left')

In [ ]:
ratings_books=ratings_books[(ratings_books['Book-Title'].notna())]

In [ ]:
ratings_books=ratings_books[ ratings_books['Book-Author'].notna()]
ratings_books['Book-Rating']=ratings_books['Book-Rating']/2

ratings_books.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(1031135, 7)

In [ ]:
ratings_books=ratings_books.iloc[:500000,0:]

ratings_books.head(5)

,User-ID,ISBN,Book-Rating,Book-Title,books_tags,Book-Author,Year-Of-Publication
0,276725,034545104X,0.0,Flesh Tones: A Novel,fleshtones:anovel m.j.rose,M. J. Rose,2002
1,276726,0155061224,2.5,Rites of Passage,ritesofpassage judithrae,Judith Rae,2001
2,276727,0446520802,0.0,The Notebook,thenotebook nicholassparks,Nicholas Sparks,1996
3,276729,052165615X,1.5,Help!: Level 1,help!:level1 philipprowse,Philip Prowse,1999
4,276729,0521795028,3.0,The Amsterdam Connection : Level 4 (Cambridge ...,theamsterdamconnection:level4(cambridgeenglish...,Sue Leather,2001


In [ ]:
ratings_books.isna().sum()

User-ID                0
ISBN                   0
Book-Rating            0
Book-Title             0
books_tags             0
Book-Author            0
Year-Of-Publication    0
dtype: int64

In [ ]:
ratings_books.rename(columns = {'User-ID':'userId', 'ISBN':'bookID','Book-Rating':'rating'}, inplace = True)
ratings_books

,userId,bookID,rating,Book-Title,books_tags,Book-Author,Year-Of-Publication
0,276725,034545104X,0.0,Flesh Tones: A Novel,fleshtones:anovel m.j.rose,M. J. Rose,2002
1,276726,0155061224,2.5,Rites of Passage,ritesofpassage judithrae,Judith Rae,2001
2,276727,0446520802,0.0,The Notebook,thenotebook nicholassparks,Nicholas Sparks,1996
3,276729,052165615X,1.5,Help!: Level 1,help!:level1 philipprowse,Philip Prowse,1999
4,276729,0521795028,3.0,The Amsterdam Connection : Level 4 (Cambridge ...,theamsterdamconnection:level4(cambridgeenglish...,Sue Leather,2001
...,...,...,...,...,...,...,...
558134,134556,0380708892,4.5,"Merlin (The Pendragon Cycle , Book 2)","merlin(thependragoncycle,book2) stephenr.lawhead",Stephen R. Lawhead,1990
558135,134556,0380708906,4.0,"Arthur (The Pendragon Cycle, Book 3)","arthur(thependragoncycle,book3) stephenr.lawhead",Stephen R. Lawhead,1990
558136,134556,0380717573,3.5,Pendragon : Book Four of the Pendragon Cycle (...,pendragon:bookfourofthependragoncycle(pendrago...,Stephen R. Lawhead,1995
558137,134556,0380781042,3.5,"Grail (The Pendragon Cycle, Book 5)","grail(thependragoncycle,book5) stephenr.lawhead",Stephen R. Lawhead,1998


In [ ]:
threshold_date = '2003' 
ratings_books['Year-Of-Publication']=pd.to_numeric(ratings_books['Year-Of-Publication'], errors='coerce')
ids = ratings_books[ratings_books['Year-Of-Publication'] < int(threshold_date)]['bookID'].values
ratings_books=ratings_books[ ratings_books['Year-Of-Publication'].notna()]
ratings_books['Year-Of-Publication']=ratings_books['Year-Of-Publication'].astype(int)
training_data = ratings_books[ratings_books['bookID'].isin(ids)]
print(f'Training data size: {training_data.shape}')
testing_data = ratings_books[~ratings_books['bookID'].isin(ids)]
print(f'Testing data size: {testing_data.shape}')
testing_data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Training data size: (450776, 7)
Testing data size: (49223, 7)


,userId,bookID,rating,Book-Title,books_tags,Book-Author,Year-Of-Publication
16,276747,0060517794,4.5,Little Altars Everywhere,littlealtarseverywhere rebeccawells,Rebecca Wells,2003
32,276762,0380000059,0.0,CLOUT,clout d.gibbons,D. GIBBONS,2024
35,276762,0451454952,0.0,House of the Sun (Shadowrun),houseofthesun(shadowrun) fanpro,Fanpro,2003
50,276762,3453877241,0.0,Live aus Bagdad. Das Tagebuch einer Kriegs-Rep...,liveausbagdad.dastagebucheinerkriegs-reporteri...,Antonia Rados,2003
74,276786,842333533X,0.0,Vieja Nueva York,viejanuevayork edithwharton,Edith Wharton,2003
...,...,...,...,...,...,...,...
558082,134516,1587670585,0.0,Oblivion,oblivion jayr.bonansinga,Jay R. Bonansinga,2004
558109,134529,0156027321,4.0,Life of Pi,lifeofpi yannmartel,Yann Martel,2003
558110,134529,0399528814,3.5,Whose Kids Are These Anyway?: True Confessions...,whosekidsaretheseanyway?:trueconfessionsofafam...,Ken Swarner,2003
558111,134529,043935806X,5.0,Harry Potter and the Order of the Phoenix (Boo...,harrypotterandtheorderofthephoenix(book5) j.k....,J. K. Rowling,2003


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(ratings_books['books_tags'])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def normalize(value, old_max, old_min, new_max=5.0, new_min=0.0):
  old_range = (old_max - old_min)  
  new_range = (new_max - new_min)  
  return (((value - old_min) * new_range) / old_range) + new_min

In [ ]:
count_matrix.shape


(499999, 269233)

In [ ]:
ratings_books.isna().sum()

userId                 0
bookID                 0
rating                 0
Book-Title             0
books_tags             0
Book-Author            0
Year-Of-Publication    0
dtype: int64

###Trining

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
class ContentBasedRecommender:
    
    MODEL_NAME = 'Content-Based'
    
    def __init__(self, items_ids, items_matrix, training_data, testing_data):
        self.items_ids = items_ids
        self.items_matrix = items_matrix
        self.training_data = training_data
        self.testing_data = testing_data
        
    def get_model_name(self):
        return self.MODEL_NAME

    def get_item_profile(self, item_id):
        idx = self.items_ids.index(item_id)
        return self.items_matrix[idx].toarray().reshape(-1)

    def get_items_profiles(self, ids):
        items_profiles = np.array([self.get_item_profile(x) for x in ids])
        return items_profiles

    def build_users_profile(self, user_id):
        user_df = self.training_data[self.training_data['userId'] == user_id]
        user_items_profiles = self.get_items_profiles(user_df['bookID'].values)
        
        user_items_ratings = np.array(user_df['rating'].values).reshape(-1,1)
        user_profile = np.sum(np.multiply(user_items_profiles, user_items_ratings), axis=0) / np.sum(user_items_ratings)
        return user_profile

    def get_similar_items_to_user_profile(self, user_id, topn=1000):
        user_profile = self.build_users_profile(user_id).reshape(1, -1)
        #Computes the cosine similarity between the user profile and all item profiles
        cosine_similarities=0
        if (len(user_profile)>0 and len(self.items_matrix.toarray())>0):
            cosine_similarities = cosine_similarity(user_profile, self.items_matrix.toarray())
        #Gets the top similar items
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        #Sort the similar items by similarity
        similar_items = sorted([(self.items_ids[i], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])
        return similar_items
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10):
        similar_items = self.get_similar_items_to_user_profile(user_id)
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        
        recommendations_df = pd.DataFrame(similar_items_filtered, columns=['bookID', 'predicted_rating']) \
                                    .head(topn)

        recommendations_df['predicted_rating'] = recommendations_df['predicted_rating'].apply(lambda x: normalize(x, 1.0, 0.0))
        return recommendations_df
    
content_based_recommender_model = ContentBasedRecommender(ratings_books['bookID'].values.tolist(), count_matrix, training_data, testing_data)
content_based_recommender_model

In [ ]:
model_evaluator = ModelEvaluator(training_data, testing_data)              


Index(['userId', 'bookID', 'rating', 'Book-Title', 'books_tags', 'Book-Author',
       'Year-Of-Publication'],
      dtype='object')


In [ ]:
print('Evaluating Content-Based Filtering model number 2...')
cb_global_metrics, cb_detailed_results_df = model_evaluator.evaluate_model(content_based_recommender_model)
print('\nGlobal metrics:\n%s' % cb_global_metrics) 

Evaluating Content-Based Filtering model number 2...
Index(['userId', 'bookID', 'rating', 'Book-Title', 'books_tags', 'Book-Author',
       'Year-Of-Publication'],
      dtype='object')


In [ ]:
cb_detailed_results_df.head(5)
 

,bookID,ratings_mean,ratings_count,predicted_rating
3398,1400032717,4.111111,9,3.446656
4028,1844262553,4.285714,7,3.436496
1505,0399149821,4.333333,6,3.377180
2054,0451526341,3.714286,14,3.319040
2640,0743454529,5.000000,3,3.243570
706,0316710571,4.500000,4,3.211094
559,0312316690,4.666667,3,3.089724
1819,0446531421,4.666667,3,3.089724
746,0345410998,4.250000,4,3.077761
297,0061015725,3.366667,15,3.058552


##Collaborative.

In [ ]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 4.4 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633733 sha256=1403e964a3a9bb65e92d7a4380016b22575f38b665e2b3166c0d1d5d6afc5db6
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
from collections import defaultdict

from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split, cross_validate

import pandas as pd

In [ ]:
Booksdata = pd.read_csv("/content/drive/MyDrive/Kuggle/Books.csv")
Usersdata = pd.read_csv("/content/drive/MyDrive/Kuggle/Users.csv")
Ratingsdata = pd.read_csv("/content/drive/MyDrive/Kuggle/Ratings.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


###  data Pre Prrocess


In [ ]:
Ratingsdata.head(10)
Ratingsdata.dropna()
mapped = Ratingsdata['ISBN'].map({v: k for k, v in Booksdata['ISBN'].items()}) 
 
 

In [ ]:
Ratingsdata['bookID']=mapped

In [ ]:
Ratingsdata.shape

(1149780, 4)

In [ ]:
Ratingsdata=Ratingsdata.dropna()


In [ ]:
Ratingsdata.bookID=Ratingsdata.bookID.astype('int64')
Ratingsdata.head() 

,User-ID,ISBN,Book-Rating,bookID
0,276725,034545104X,0,2966
1,276726,0155061224,5,225816
2,276727,0446520802,0,11053
3,276729,052165615X,3,246838
4,276729,0521795028,6,246839


In [ ]:
reader = Reader(rating_scale=(0, 10))
Ratingsdata.isna().sum()

User-ID        0
ISBN           0
Book-Rating    0
bookID         0
dtype: int64

In [ ]:
reader = Reader(rating_scale=(0, 10))
surprise_data = Dataset.load_from_df(Ratingsdata[['User-ID', 'ISBN', 'Book-Rating']], reader)
trainset, testset = train_test_split(surprise_data, test_size=0.25)

### 2 Taining


In [ ]:
benchmark = [] 
for algorithm in [SVD()]: 
    results = cross_validate(algorithm, surprise_data, measures=['RMSE'], cv=3, verbose=False)
     
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)

In [ ]:
svd = SVD() 
svd.fit(trainset)

In [ ]:
from collections import defaultdict

from surprise import Dataset
from surprise import SVD
from surprise.model_selection import KFold


def precision_recall_at_k(predictions, k=10, threshold=3.5): 
    print (predictions[1])
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():
 
        user_ratings.sort(key=lambda x: x[0], reverse=True)
 
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
 
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
 
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])
 
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
 
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0
        print('recall',recalls)
    return precisions, recalls




In [ ]:
predictions = svd.test(testset)

In [ ]:

precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4)

print(sum(prec for prec in precisions.values()) / len(precisions))
print(sum(rec for rec in recalls.values()) / len(recalls))

user: 164944     item: 2840702266 r_ui = 0.00   est = 0.88   {'was_impossible': False}
0.1993519371060283
0.16428470580542184


In [ ]:
predictions = algo.test(testset)

In [ ]:
precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4)


user: 228998     item: 167100     r_ui = 0.00   est = 0.57   {'was_impossible': False}
